# Flight Price Single-Day Exploratory Analysis & PyTorch Pipeline

This notebook performs extensive exploratory data analysis (EDA) and builds a PyTorch-based preprocessing and modeling pipeline on a single-day subset of flight pricing data. It follows core data mining course techniques and augments them with advanced, production-oriented practices.

In [2]:
# Section 1: Load Packages and Set Global Config
from __future__ import annotations
import os, sys, math, json, random, pathlib, itertools, statistics
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Tuple, Optional

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Optional: set pandas display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 160)

# Global seed function
DEFAULT_SEED = 42

def set_all_seeds(seed: int = DEFAULT_SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds()

# Basic path setup
DATA_PATH = pathlib.Path('../data/turncated/flightDate_2022-11-11.csv').resolve()
print(f"Using data file: {DATA_PATH}")


Using data file: /Users/I568528/Documents/projects/Uni Projects/Data-Mining/data_mining_flight/data/turncated/flightDate_2022-11-11.csv


In [3]:
# Section 2: Ingest CSV Subset and Memory-Efficient Read Options
# Define dtype hints for initial load (opt for object for complex multi-segment fields)
dtype_map = {
    'legId': 'string',
    'searchDate': 'string',
    'flightDate': 'string',
    'startingAirport': 'category',
    'destinationAirport': 'category',
    'fareBasisCode': 'string',
    'travelDuration': 'string',  # ISO 8601 duration
    'elapsedDays': 'int16',
    'isBasicEconomy': 'bool',
    'isRefundable': 'bool',
    'isNonStop': 'bool',
    'baseFare': 'float32',
    'totalFare': 'float32',
    'seatsRemaining': 'float32',  # may contain 0 or missing
    'totalTravelDistance': 'float32',
    'segmentsDepartureTimeEpochSeconds': 'string',
    'segmentsDepartureTimeRaw': 'string',
    'segmentsArrivalTimeEpochSeconds': 'string',
    'segmentsArrivalTimeRaw': 'string',
    'segmentsArrivalAirportCode': 'string',
    'segmentsDepartureAirportCode': 'string',
    'segmentsAirlineName': 'string',
    'segmentsAirlineCode': 'string',
    'segmentsEquipmentDescription': 'string',
    'segmentsDurationInSeconds': 'string',
    'segmentsDistance': 'string',
    'segmentsCabinCode': 'string'
}

# Read full dataset (single day subset)
df = pd.read_csv(DATA_PATH, dtype=dtype_map)
print(f"Loaded shape: {df.shape}")
print(f"Estimated memory usage: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")

# Duplicate legId check
dup_legids = df['legId'].duplicated().sum()
print(f"Duplicate legId count: {dup_legids}")


Loaded shape: (101269, 27)
Estimated memory usage: 132.50 MB
Duplicate legId count: 80363


In [4]:
# Section 3: Quick Raw Inspection and Data Type Summary
print(df.head())
print(df.tail(2))
print(df.sample(3, random_state=DEFAULT_SEED))
print(df.info())
print(df[['startingAirport','destinationAirport']].describe())
print(df['totalFare'].describe())
print(df['baseFare'].describe())
print(df['seatsRemaining'].describe())

# Value counts for key categorical-like fields
for col in ['segmentsAirlineName', 'segmentsAirlineCode', 'fareBasisCode']:
    if col in df.columns:
        print(f"\nValue counts for {col} (top 10):")
        print(df[col].value_counts().head(10))

# Section 4: Construct Data Dictionary (Automated Schema Inference)
feature_roles = {}
for c in df.columns:
    if c == 'totalFare':
        role = 'target'
    elif c == 'legId':
        role = 'id'
    elif c in ['searchDate','flightDate']:
        role = 'temporal'
    elif 'segments' in c:
        role = 'multi_segment'
    else:
        role = 'feature'
    feature_roles[c] = role

schema_records = [
    {
        'name': c,
        'role': feature_roles[c],
        'dtype': str(df[c].dtype),
        'n_unique': df[c].nunique(dropna=True),
        'pct_missing': float(df[c].isna().mean()),
    }
    for c in df.columns
]
print("Schema sample:")
print(schema_records[:5])

# Section 5: Parse and Clean Columns (Durations, Epochs, Multi-Segment Fields)
import re

def parse_duration(iso_text: str) -> Optional[int]:
    if not isinstance(iso_text, str) or not iso_text.startswith('PT'):
        return None
    # Pattern PT#H#M#S optional parts
    hours = minutes = seconds = 0
    h_match = re.search(r'(\d+)H', iso_text)
    m_match = re.search(r'(\d+)M', iso_text)
    s_match = re.search(r'(\d+)S', iso_text)
    if h_match: hours = int(h_match.group(1))
    if m_match: minutes = int(m_match.group(1))
    if s_match: seconds = int(s_match.group(1))
    return hours*60 + minutes + (seconds/60)

# Apply parse to travelDuration
if 'travelDuration' in df.columns:
    df['travelDuration_minutes'] = df['travelDuration'].apply(parse_duration).astype('float32')

# Epoch conversions for departure/arrival times (first segment only for simplicity in EDA)
from datetime import datetime, timezone

def epoch_to_dt(value: str) -> Optional[pd.Timestamp]:
    if not isinstance(value,str) or value.strip()=='' or '||' in value:
        # take first if multi
        if isinstance(value,str) and '||' in value:
            value = value.split('||')[0]
        else:
            return None
    try:
        e = int(value)
        return pd.to_datetime(e, unit='s', utc=True)
    except Exception:
        return None

if 'segmentsDepartureTimeEpochSeconds' in df.columns:
    df['firstDepartureUTC'] = df['segmentsDepartureTimeEpochSeconds'].apply(epoch_to_dt)
if 'segmentsArrivalTimeEpochSeconds' in df.columns:
    df['firstArrivalUTC'] = df['segmentsArrivalTimeEpochSeconds'].apply(epoch_to_dt)

# Multi-segment splitting
segment_cols = [
    'segmentsDepartureTimeEpochSeconds','segmentsArrivalTimeEpochSeconds',
    'segmentsArrivalAirportCode','segmentsDepartureAirportCode','segmentsAirlineName',
    'segmentsAirlineCode','segmentsEquipmentDescription','segmentsDurationInSeconds',
    'segmentsDistance','segmentsCabinCode'
]
for sc in segment_cols:
    if sc in df.columns:
        df[sc + '_list'] = df[sc].apply(lambda x: x.split('||') if isinstance(x,str) and '||' in x else [x])

# Section 6: Normalize Boolean, Categorical, and Numeric Fields
bool_cols = ['isBasicEconomy','isRefundable','isNonStop']
for bc in bool_cols:
    if bc in df.columns:
        df[bc] = df[bc].astype('bool')

cat_cols = ['startingAirport','destinationAirport']
for cc in cat_cols:
    if cc in df.columns:
        df[cc] = df[cc].astype('category')

num_cols = ['baseFare','totalFare','seatsRemaining','totalTravelDistance','travelDuration_minutes']
for nc in num_cols:
    if nc in df.columns:
        df[nc] = pd.to_numeric(df[nc], errors='coerce').astype('float32')

# Section 7: Feature Engineering: Time, Distance, Price Metrics
if 'firstDepartureUTC' in df.columns:
    df['departure_hour'] = df['firstDepartureUTC'].dt.hour.astype('float32')
if 'firstArrivalUTC' in df.columns:
    df['arrival_hour'] = df['firstArrivalUTC'].dt.hour.astype('float32')
if 'firstDepartureUTC' in df.columns and 'firstArrivalUTC' in df.columns:
    df['is_overnight'] = (df['arrival_hour'] < df['departure_hour']).astype('int8')

if 'totalTravelDistance' in df.columns and 'totalFare' in df.columns:
    df['price_per_mile'] = df['totalFare'] / df['totalTravelDistance']
if 'baseFare' in df.columns and 'totalFare' in df.columns:
    df['fare_margin'] = df['totalFare'] - df['baseFare']

# Section 8: Feature Engineering: Segment Aggregations and Connection Statistics
if 'segmentsDistance_list' in df.columns:
    df['number_of_segments'] = df['segmentsDistance_list'].apply(len).astype('int16')
    def safe_float_list(lst):
        out = []
        for v in lst:
            try:
                out.append(float(v))
            except Exception:
                pass
        return out
    dist_lists = df['segmentsDistance_list'].apply(safe_float_list)
    df['total_segment_distance'] = dist_lists.apply(lambda x: sum(x) if x else np.nan).astype('float32')

if 'segmentsDurationInSeconds_list' in df.columns:
    dur_lists = df['segmentsDurationInSeconds_list'].apply(safe_float_list)
    df['avg_segment_duration_min'] = dur_lists.apply(lambda x: (sum(x)/len(x)/60.0) if x else np.nan).astype('float32')

# Connection wait times (layovers) approximation using departure/arrival raw times placeholder
# (Advanced precise computation would require parsing raw times per segment; placeholder for now)
df['longest_layover_minutes'] = np.nan  # to be computed in advanced section later

print("Engineered columns added. Current columns count:", len(df.columns))


                              legId  searchDate  flightDate startingAirport destinationAirport fareBasisCode travelDuration  elapsedDays  isBasicEconomy  \
0  80132eb7d759fd6970aeefb5d20366d4  2022-09-21  2022-11-11             ATL                BOS         RA7NR        PT2H31M            0           False   
1  0e853c84a66b0c781d378c879d4f06a9  2022-09-21  2022-11-11             ATL                BOS       W00PXP4       PT11H57M            1           False   
2  a4623d2af8f5557e56bf988c58237d5c  2022-09-21  2022-11-11             ATL                BOS      PI4QUEL1        PT2H28M            0           False   
3  f57876a19f10414445c02c5e45f307de  2022-09-21  2022-11-11             ATL                BOS      KAA4AWBN           PT4H            0            True   
4  d4d489a031733140fd69bf40ce989249  2022-09-21  2022-11-11             ATL                BOS      KAA4AWBN        PT5H14M            0            True   

   isRefundable  isNonStop   baseFare   totalFare  seatsRemaini